# Imports

In [1]:
import pandas as pd
import numpy as np
from random import shuffle
from tqdm.notebook import tqdm
import time
from math import ceil
import csv
import sys

# Random dataset

In [2]:
np.random.seed(42)

In [3]:
nodes = 4
# creating a random matrix with respective probabilities
val = np.random.choice(2, size=(nodes, nodes), p=[0.7, 0.3])
val

array([[0, 1, 1, 0],
       [0, 0, 0, 1],
       [0, 1, 0, 1],
       [1, 0, 0, 0]])

In [ ]:
# convert the random matrix to a adjacency matrix by make it symmetric to diagonal
for i, row in enumerate(val):
  val[:, i] = row
np.fill_diagonal(val, 1)
data = val

# Real Data

In [23]:
data = pd.read_excel('new_300_rows.xlsx')
data = np.array(data)
data[data == -1] = 0
nodes = len(data)
data = data.astype(int)

In [24]:
sum(sum(data))/300 ** 2

0.31462222222222225

# Utility functions

In [2]:
def create_hash_func(size: int):
    # function for creating the hash vector/function
    hash_ex = list(range(size))
    shuffle(hash_ex)
    return hash_ex

def build_minhash_func(vocab_size: int, nbits: int):
    # function for building multiple minhash vectors
    hashes = []
    for _ in range(nbits):
        hashes.append(create_hash_func(vocab_size))
    return hashes

In [3]:
def create_hash(minhash_fn, vector: list):
    # use this function for creating our signatures (eg the matching)
    signature = []
    for func in minhash_fn:
        for i in range(len(vector)):
            idx = func.index(i)
            signature_val = vector[idx]
            if signature_val == 1:
                signature.append(idx)
                break
    return signature

In [4]:
def load_dataset(ds_name):
    # read dataset and turn to np array
    data = pd.read_excel(ds_name)
    data = np.array(data)

    # dissimlar edges as no edge
    data[data == -1] = 0

    data = data.astype(int)

    return data

In [13]:
with open("new_300_rows.csv") as dataset:
    # for i in
    # Create a CSV reader object
    reader = csv.reader(dataset)
    
    # Read a single line from the file
    line = next(reader)
    # G
    size = sys.getsizeof(line)
    print(size, sys.getsizeof(reader))
    # Convert the line to a list
    # line_list = list(line)stri
    # line_list[0] = line_list[0].p('\ufeff')
    line_list = list(map(int,list(line)))
    print(line_list)

2520 96
[1, -1, -1, -1, 1, -1, -1, 1, -1, -1, -1, 1, 1, 1, 1, -1, -1, -1, -1, 1, 1, -1, -1, 1, 1, -1, 1, -1, -1, 1, -1, -1, -1, -1, -1, -1, -1, 1, 1, -1, -1, -1, -1, -1, 1, -1, -1, -1, -1, 1, -1, -1, -1, 1, -1, 1, -1, -1, -1, 1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, 1, -1, 1, 1, 1, -1, 1, 1, -1, -1, -1, -1, -1, 1, 1, -1, -1, -1, -1, -1, -1, 1, 1, -1, -1, -1, 1, 1, -1, -1, 1, -1, -1, 1, -1, 1, -1, -1, -1, -1, -1, -1, -1, -1, 1, -1, 1, -1, 1, -1, 1, -1, 1, 1, 1, -1, -1, -1, 1, -1, 1, -1, 1, -1, 1, 1, -1, -1, -1, 1, -1, 1, 1, -1, 1, 1, -1, 1, -1, -1, 1, 1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, 1, -1, -1, 1, -1, -1, -1, 1, 1, 1, -1, -1, -1, 1, -1, -1, -1, -1, -1, -1, -1, 1, -1, 1, -1, -1, -1, -1, -1, 1, -1, -1, 1, -1, 1, -1, -1, 1, -1, 1, -1, 1, -1, -1, 1, -1, -1, -1, -1, -1, 1, -1, -1, -1, -1, 1, -1, -1, -1, -1, -1, 1, 1, 1, 1, -1, -1, -1, -1, -1, -1, -1, -1, -1, 1, 1, -1, -1, -1, 1, 1, -1, 1, -1, -1, -1, -1, 1, -1, 1, -1, -1, -1, -1, -1, -1, -1, 1, -1, -1, -1, 1, -1, 1, 1

In [8]:
8 * 300

2400

# Score Functions

In [5]:
def fast_score(data, cluster):
    score = 0
    nodes = len(data)
    for i in (range(nodes)):
        for j in range(i + 1, nodes):
            # if i and j are similar and not in a same cluster
            if data[i][j] == 1:
                # and not in the same cluster
                if cluster[i] != cluster[j]: score -= 1
            # else if they are not similar
            else:
                # and in the same cluster
                if cluster[i] == cluster[j]: score -= 1

    return score
             

In [6]:
def approx_score(data, cluster):

    score = 0
    nodes = len(data)
    nodes_sqrt = int(nodes ** (1/2))

    # a perm. of size n
    permutation = np.arange(nodes_sqrt).astype(int)
    shuffle(permutation)
    

    for i in permutation:
        for j in range(nodes):
            # if i and j are similar and not in a same cluster
            if data[i][j] == 1:
                # and not in the same cluster
                
                if cluster[i] != cluster[j]: score -= 1
            # else if they are not similar
            else:
                # and in the same cluster
                if cluster[i] == cluster[j]: score -= 1

    return score * nodes_sqrt

In [7]:
def find_best_approx_cluster(bands_clusters):
  max = -float('inf')
  max_cluster= None
  for cluster in bands_clusters:
    # scr = score(data, cluster)
    scr = approx_score(data, cluster)
    if scr > max:
      max = scr
      max_cluster = cluster
    if scr == 0:
      break
  return max_cluster, max

In [8]:
def find_best_cluster(bands_clusters):
  max = -float('inf')
  max_cluster= None
  for cluster in bands_clusters:
    # scr = score(data, cluster)
    scr = fast_score(data, cluster)
    if scr > max:
      max = scr
      max_cluster = cluster
    if scr == 0:
      break
  return max_cluster, max

# Lsh functions

In [9]:
def regular_approx_LSH(num_iter, data, row_num, band_num, sigs_num):
  #initialize
  best_cluster = None
  best_score = -float('inf')
  nodes = len(data)

  for _ in tqdm(range(num_iter)):
    # we create 20 minhash vectors
    minhash_funcs = build_minhash_func(len(data), sigs_num)

    # now create signatures
    sigs = []
    for i in range(len(data)):
      sigs.append(create_hash(minhash_funcs, data[i]))
    # print(len(sigs), len(sigs[0]))
    bands_clusters = []
    # print(band_num)
    for i in range(band_num):
      band_cluster = [0] * nodes
      for j, sig in enumerate(sigs):
        # print(''.join(map(str,sig[i*row_num: (i+1)*row_num])))
        # print(sig[i*row_num: (i+1)*row_num])
        lsh = int(''.join(map(str,sig[i*row_num: (i+1)*row_num])), 10) #% nodes

        if len(sig[i*row_num: (i+1)*row_num]) == 0: break
        band_cluster[j] = lsh
      bands_clusters.append(band_cluster)
    cluster, scr = find_best_approx_cluster(bands_clusters)
    if scr > best_score:
      best_score = scr
      best_cluster = cluster
  return fast_score(data, best_cluster)


In [64]:
def regular_LSH(num_iter, dataset_name, dataset_length, row_num, band_num, sigs_num):
    #initialize
    best_cluster = None
    best_score = -float('inf')
    nodes = dataset_length

    for _ in tqdm(range(num_iter)):
        # Create minhash sig_num vectors(e.g 30)
        minhash_funcs = build_minhash_func(dataset_length, sigs_num)

        # List of signitures
        sigs = []

        # List of clusters for each band
        bands_clusters = []

        # List of scores for each cluster
        bands_clusters_score = [0] * band_num

        # Initialize
        for i in range(band_num):
            bands_clusters.append([0] * dataset_length)

        with open(dataset_name) as dataset:
            # Create a CSV reader object
            reader = csv.reader(dataset)

            for i in range(dataset_length):
                
                # Read a single line from the file which is similarities of one node to every one
                line = next(reader)

                # Convert to a list of ints
                line_list = list(map(int,list(line)))
                # print(line_list, i)
                # Create signature for i'th node
                sig = create_hash(minhash_funcs, line_list)

                # For each band
                for b in range(band_num):
                    # H ash the band
                    lsh = int(''.join(map(str,sig[b*row_num: (b+1)*row_num])), 10) % nodes

                    # Now we know the cluster for node i in band b
                    bands_clusters[b][i] = lsh

            # Now we have b clusters and want to find the best of them
            dataset.seek(0)
            # Create a CSV reader object
            reader = csv.reader(dataset)

            for i in range(dataset_length):
        
                # Read a single line from the file which is similarities of one node to every one
                line = next(reader)

                # Convert to a list of ints
                line_list = list(map(int,list(line)))

                # for each node ahead of the i'th node
                for j in range(i + 1, dataset_length):
                    
                    # and not in the same cluster
                    for b in range(band_num):

                        # If i and j are in different clusters
                        if bands_clusters[b][i] != bands_clusters[b][j]:

                            # If i and j are similar
                            if line_list[j] == 1:
                                bands_clusters_score[b] -= 1

                        # If i and j are in the same cluster
                        else:

                            # If i and j are dissimilar
                            if line_list[j] == -1:
                                bands_clusters_score[b] -= 1
        scr = max(bands_clusters_score)

        if scr > best_score:
            best_score = scr
            
    return best_score


In [74]:
from random import shuffle
def random_row_LSH(num_iter, dataset_name, dataset_length, k, band_num=1):
    #initialize
    # best_cluster = None
    best_score = -float('inf')
    nodes = dataset_length

    for _ in tqdm(range(num_iter)):
        # Create minhash sig_num vectors(e.g 30)
        # minhash_funcs = build_minhash_func(dataset_length, sigs_num)


        # # List of clusters for each band
        # bands_clusters = []

        # # List of scores for each cluster
        # bands_clusters_score = [0] * band_num

        cluster = [0] * dataset_length
        score = 0

        # # Initialize
        # for i in range(band_num):
        #     bands_clusters.append([0] * dataset_length)
        perm = list(range(dataset_length))
        shuffle(perm)
        perm = perm[:k]
        
        with open(dataset_name) as dataset:
            # Create a CSV reader object
            reader = csv.reader(dataset)

            for i in range(dataset_length):
                
                # Read a single line from the file which is similarities of one node to every one
                line = next(reader)

                # If i not in the permutations go to the next i
                # if i not in perm: continue

                # Convert to a list of ints
                line_list = list(map(int,list(line)))
                
                # Create signature for i'th node
                sig = [line_list[i] if line_list[i] == 1 else 0 for i in perm]

                # For each band
                for b in range(band_num):
                    # H ash the band
                    lsh = int(''.join(map(str,sig)), 10) % nodes

                    # Now we know the cluster for node i in band b
                    cluster[i] = lsh

            # Now we have b clusters and want to find the best of them
            dataset.seek(0)
            # Create a CSV reader object
            reader = csv.reader(dataset)

            for i in range(dataset_length):
        
                # Read a single line from the file which is similarities of one node to every one
                line = next(reader)

                # Convert to a list of ints
                line_list = list(map(int,list(line)))

                # for each node ahead of the i'th node
                for j in range(i + 1, dataset_length):
                    
                    # and not in the same cluster
                    for b in range(band_num):

                        # If i and j are in different clusters
                        if cluster[i] != cluster[j]:

                            # If i and j are similar
                            if line_list[j] == 1:
                                score -= 1

                        # If i and j are in the same cluster
                        else:

                            # If i and j are dissimilar
                            if line_list[j] == -1:
                                score -= 1
        # scr = max(score)

        if score > best_score:
            best_score = score
            
    return best_score


In [7]:
def nlp_regular_LSH(num_iter, dataset_name, dataset_length, row_num, band_num, sigs_num):
    #initialize
    best_cluster = None
    best_score = -float('inf')
    nodes = dataset_length

    for _ in tqdm(range(num_iter)):
        # Create minhash sig_num vectors(e.g 30)
        minhash_funcs = build_minhash_func(dataset_length, sigs_num)

        # List of signitures
        sigs = []

        # List of clusters for each band
        bands_clusters = []

        # List of scores for each cluster
        bands_clusters_score = [0] * band_num

        # Initialize
        for i in range(band_num):
            bands_clusters.append([0] * dataset_length)

        dataset = np.load('adj_mat_100.npy')

        # with open(dataset_name) as dataset:
        
            # Create a CSV reader object
            # reader = csv.reader(dataset)

            # for i in range(dataset_length):
        for i, line in enumerate(dataset):
            # Read a single line from the file which is similarities of one node to every one
            # line = next(reader)
            # print(line)
            # Convert to a list of ints
            # line_list = list(map(int,list(line)))
            # print(line_list)
            # break
            # print(line_list, i)
            # Create signature for i'th node
            sig = create_hash(minhash_funcs, line)

            # For each band
            for b in range(band_num):
                # H ash the band
                lsh = int(''.join(map(str,sig[b*row_num: (b+1)*row_num])), 10)
                # print(lsh)
                lsh = lsh % 2
                # print(lsh)
                # Now we know the cluster for node i in band b
                bands_clusters[b][i] = lsh

            # Now we have b clusters and want to find the best of them
            # dataset.seek(0)
            # Create a CSV reader object
            # reader = csv.reader(dataset)

            for i,line in enumerate(dataset):
                # print(line)
                # Read a single line from the file which is similarities of one node to every one
                # line = next(reader)

                # # Convert to a list of ints
                # line_list = list(map(int,list(line)))

                # for each node ahead of the i'th node
                for j in range(i + 1, dataset_length):
                    
                    # and not in the same cluster
                    for b in range(band_num):

                        # If i and j are in different clusters
                        if bands_clusters[b][i] != bands_clusters[b][j]:

                            # If i and j are similar
                            if line[j] == 1:
                                bands_clusters_score[b] -= 1

                        # If i and j are in the same cluster
                        else:

                            # If i and j are dissimilar
                            if line[j] == -1:
                                bands_clusters_score[b] -= 1
        scr = max(bands_clusters_score)
        idx = [i for i in range(band_num) if bands_clusters_score[i]== scr][0]
        cluster = bands_clusters[idx]
        # print(bands_clusters)
        # print(idx, 'dfdf')
        if scr > best_score:
            best_score = scr
            best_cluster = cluster
            
    return best_score, best_cluster


In [14]:
length = 100
ds_name = 'new_'+str(length)+'_rows.csv'
total_num_sigs= 30
num_iter = 1
node_log = np.floor(np.log2(total_num_sigs)).astype(int)
row_num = 10
band_num = ceil(total_num_sigs / row_num)
# regular_LSH(num_iter, data, row_num, band_num, sigs_num)
score, cluster = nlp_regular_LSH(num_iter, 'mat_100.csv', length, row_num, band_num, total_num_sigs)
# output_log = 'dataset: '+ ds_name +' \n score: '+ str(score) +' \n band: '+ str(band_num) + ' \n number of signitures: ' +str(j)+'\n\n'
# print(output_log)
print('score:', score)
print(cluster)
print(sum(cluster == target))

  0%|          | 0/1 [00:00<?, ?it/s]

score: -11810
[0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1]
82


In [10]:
target = np.load('tgt_100.npy')

In [142]:
target

array([0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1,
       0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1], dtype=int64)

In [ ]:
cluster

In [141]:
sum(cluster == target)

66

66

# LSH

In [77]:
length = 1000
ds_name = 'new_'+str(length)+'_rows.csv'
total_num_sigs= 1000
num_iter = 10
node_log = np.floor(np.log2(length)).astype(int)
row_num = node_log
band_num = ceil(total_num_sigs / row_num)
# regular_LSH(num_iter, data, row_num, band_num, sigs_num)
score = random_row_LSH(num_iter, 'mat_100.csv', length, k =total_num_sigs)
# output_log = 'dataset: '+ ds_name +' \n score: '+ str(score) +' \n band: '+ str(band_num) + ' \n number of signitures: ' +str(j)+'\n\n'
# print(output_log)
print('score:', score)

  0%|          | 0/10 [00:00<?, ?it/s]

ValueError: invalid literal for int() with base 10: '1.000000000000000000e+00'

In [ ]:
def Parallel_regular_LSH(num_iter, dataset_name, dataset_length, row_num, band_num, sigs_num):
    #initialize
    best_cluster = None
    best_score = -float('inf')
    nodes = dataset_length

    for _ in tqdm(range(num_iter)):
        # Create minhash sig_num vectors(e.g 30)
        minhash_funcs = build_minhash_func(dataset_length, sigs_num)

        # List of signitures
        sigs = []

        # List of clusters for each band
        bands_clusters = []

        # List of scores for each cluster
        bands_clusters_score = [0] * band_num

        # Initialize
        for i in range(band_num):
            bands_clusters.append([0] * dataset_length)

        with open(dataset_name) as dataset:
            # Create a CSV reader object
            reader = csv.reader(dataset)

            for i in range(dataset_length):
                
                # Read a single line from the file which is similarities of one node to every one
                line = next(reader)

                # Convert to a list of ints
                line_list = list(map(int,list(line)))
                # print(line_list, i)
                # Create signature for i'th node
                sig = create_hash(minhash_funcs, line_list)

                # For each band
                for b in range(band_num):
                    # H ash the band
                    lsh = int(''.join(map(str,sig[b*row_num: (b+1)*row_num])), 10) % nodes

                    # Now we know the cluster for node i in band b
                    bands_clusters[b][i] = lsh

            # Now we have b clusters and want to find the best of them
            dataset.seek(0)
            # Create a CSV reader object
            reader = csv.reader(dataset)

            for i in range(dataset_length):
        
                # Read a single line from the file which is similarities of one node to every one
                line = next(reader)

                # Convert to a list of ints
                line_list = list(map(int,list(line)))

                # for each node ahead of the i'th node
                for j in range(i + 1, dataset_length):
                    
                    # and not in the same cluster
                    for b in range(band_num):

                        # If i and j are in different clusters
                        if bands_clusters[b][i] != bands_clusters[b][j]:

                            # If i and j are similar
                            if line_list[j] == 1:
                                bands_clusters_score[b] -= 1

                        # If i and j are in the same cluster
                        else:

                            # If i and j are dissimilar
                            if line_list[j] == -1:
                                bands_clusters_score[b] -= 1
        scr = max(bands_clusters_score)

        if scr > best_score:
            best_score = scr
            
    return best_score


In [65]:
length = 1000
ds_name = 'new_'+str(length)+'_rows.csv'
total_num_sigs= 30
num_iter = 1
node_log = np.floor(np.log2(length)).astype(int)
row_num = node_log
band_num = ceil(total_num_sigs / row_num)
# regular_LSH(num_iter, data, row_num, band_num, sigs_num)
score = regular_LSH(num_iter, ds_name, length, row_num, band_num, total_num_sigs)
# output_log = 'dataset: '+ ds_name +' \n score: '+ str(score) +' \n band: '+ str(band_num) + ' \n number of signitures: ' +str(j)+'\n\n'
# print(output_log)
print('score:', score)

  0%|          | 0/1 [00:00<?, ?it/s]

score: -74623
